In [31]:
import os
import numpy as np
import time
import statistics
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import ShuffleSplit
import pathlib
import cv2

In [32]:
data_dir = './MP_image/'

data_dir=pathlib.Path(data_dir)
data_dir

WindowsPath('MP_image')

In [33]:
MP_image_dict = {
    'days_28b': list(data_dir.glob('before28days/*')),
    'days_28a': list(data_dir.glob('after28days/*'))
    
}

In [34]:
MP_label_dict={
    'days_28b': 0,
    'days_28a': 1
}

In [35]:
X ,y =[], []

for aged_days, images in MP_image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_image = cv2.resize(img, (224,224))
        X.append(resized_image)
        y.append(MP_label_dict[aged_days])

In [36]:
X = np.array(X)
y = np.array(y)
y.shape

(131,)

In [37]:
x=X.astype('float32')/255.0
x.shape

(131, 224, 224, 3)

In [38]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

In [39]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

SPLITS = 50

# Bootstrap
boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1, 
                                random_state=42)

# Track progress
mean_benchmark = []
epochs_needed = []
num = 0

# Loop through samples
for train, test in boot.split(x,y):
    start_time = time.time()
    num+=1

    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Construct neural network
    model = Sequential([
        layers.Conv2D(16,kernel_size=(3,3), padding='same', activation='relu', input_shape=(224,224,3)),
        layers.MaxPooling2D(),
        layers.Conv2D(32,kernel_size=(3,3), padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64,kernel_size=(3,3), padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='softmax')])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=25, verbose=0, mode='auto', restore_best_weights=True)

    # Train on the bootstrap sample
    model.fit(x_train,y_train,validation_data=(x_test,y_test),
              callbacks=[monitor],verbose=0,epochs=1000)
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)
    
    # Predict on the out of boot (validation)
    pred = model.predict(x_test)
  
    # Measure this bootstrap's log loss
    y_compare = np.argmax(y_test,axis=1) # For log loss calculation
    score = metrics.log_loss(y_compare, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)
    
    # Record this iteration
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f}," +\
          f"stdev={mdev:.6f}, epochs={epochs}, mean epochs={int(m2)}," +\
          f" time={hms_string(time_took)}")

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Relu (defined at <ipython-input-39-7c618a0ec620>:41) ]] [Op:__inference_train_function_691]

Function call stack:
train_function


In [26]:
x_train.shape

(118, 224, 224, 3)

In [28]:
y_train.shape

(118, 6)

In [29]:
x_test.shape

(14, 224, 224, 3)

In [30]:
y_test.shape

(14, 6)